In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"alanjacob03","key":"1b8ab587f0d6b7e7e08650d6e6708308"}'}

In [ ]:
!mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d amanneo/diabetic-retinopathy-resized-arranged

100% 1.21G/1.21G [00:17<00:00, 24.5MB/s]
100% 1.21G/1.21G [00:17<00:00, 73.0MB/s]


In [ ]:
! mkdir Data

In [ ]:
! unzip diabetic-retinopathy-resized-arranged.zip -d Data

In [ ]:
import numpy as np
import pandas as pd
import os
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet import ResNet101
import matplotlib.pyplot as plt
import shutil
import random
from keras.layers import Conv2D, Dense, MaxPooling2D, Dropout, Flatten, GlobalAveragePooling2D
import cv2
import tensorflow as tf
import gc

In [ ]:
IMG_SIZE = 224

In [ ]:
root_path = '/content/kaggle/working/Data'
if os.path.exists(root_path):
    shutil.rmtree(root_path)

train_dir = os.makedirs(os.path.join(root_path, 'training'))
val_dir = os.makedirs(os.path.join(root_path, 'validation'))
class_names = ['0','1','2','3','4']

train_healthy = os.makedirs(os.path.join(root_path, 'training/0'))
train_mildDR = os.makedirs(os.path.join(root_path, 'training/1'))
train_moderateDR = os.makedirs(os.path.join(root_path, 'training/2'))
train_proliferateDR = os.makedirs(os.path.join(root_path, 'training/3'))
train_severeDR = os.makedirs(os.path.join(root_path, 'training/4'))

val_healthy = os.makedirs(os.path.join(root_path, 'validation/0'))
val_mildDR = os.makedirs(os.path.join(root_path, 'validation/1'))
valval_moderateDR = os.makedirs(os.path.join(root_path, 'validation/2'))
val_proliferateDR = os.makedirs(os.path.join(root_path, 'validation/3'))
val_severeDR = os.makedirs(os.path.join(root_path, 'validation/4'))

In [ ]:
source_path = '/content/Data'

In [ ]:
source_path_healthy = os.path.join(source_path, class_names[0])
source_path_mildDR = os.path.join(source_path, class_names[1])
source_path_moderateDR = os.path.join(source_path, class_names[2])
source_path_proliferateDR = os.path.join(source_path, class_names[3])
source_path_severeDR = os.path.join(source_path, class_names[4])

DR_class_count = [source_path_mildDR, source_path_moderateDR, source_path_proliferateDR, source_path_severeDR]

# os.listdir returns a list containing all files under the given path
print(f"There are {len(os.listdir(source_path_healthy))} images of 0.")
print(f"There are {len(os.listdir(source_path_mildDR))} images of 1.")
print(f"There are {len(os.listdir(source_path_moderateDR))} images of 2.")
print(f"There are {len(os.listdir(source_path_proliferateDR))} images of 3.")
print(f"There are {len(os.listdir(source_path_severeDR))} images of 4.")

In [ ]:
def split_data(SOURCE_DIR, TRAINING_DIR, VALIDATION_DIR, SPLIT_SIZE):
    all_files = []
    for fn in os.listdir(SOURCE_DIR):
        file_path = SOURCE_DIR + fn
        if os.path.getsize(file_path)>0:
            all_files.append(fn)
        else:
            print("\n" + fn + " is zero length, so ignoring.")
    n_files = len(all_files)
    split_point = int(n_files * SPLIT_SIZE)
    shuffled = random.sample(all_files, n_files)

    train_set = shuffled[:split_point]
    test_set = shuffled[split_point:]

    for file_name in train_set:
        #shutil.copyfile(SOURCE_DIR + file_name, TRAINING_DIR + file_name)
        img = cv2.imread(SOURCE_DIR + file_name)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        cv2.imwrite(TRAINING_DIR + file_name, img)

    for file_name in test_set:
        #shutil.copyfile(SOURCE_DIR + file_name, VALIDATION_DIR + file_name)
        img = cv2.imread(SOURCE_DIR + file_name)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        cv2.imwrite(VALIDATION_DIR + file_name, img)

HEALTHY_SOURCE_DIR = "/content/Data/0/"
TRAINING_HEALTHY_DIR = "/content/kaggle/working/Data/training/0/"
VALIDATION_HEALTHY_DIR = "/content/kaggle/working/Data/validation/0/"

MILDDR_SOURCE_DIR = "/content/Data/1/"
TRAINING_MILDDR_DIR = "/content/kaggle/working/Data/training/1/"
VALIDATION_MILDDR_DIR = "/content/kaggle/working/Data/validation/1/"

MODERATEDR_SOURCE_DIR = "/content/Data/2/"
TRAINING_MODERATEDR_DIR = "/content/kaggle/working/Data/training/2/"
VALIDATION_MODERATEDR_DIR = "/content/kaggle/working/Data/validation/2/"

PROLIFERATEDR_SOURCE_DIR = "/content/Data/3/"
TRAINING_PROLIFERATEDR_DIR = "/content/kaggle/working/Data/training/3/"
VALIDATION_PROLIFERATEDR_DIR = "/content/kaggle/working/Data/validation/3/"

SEVEREDR_SOURCE_DIR = "/content/Data/4/"
TRAINING_SEVEREDR_DIR = "/content/kaggle/working/Data/training/4/"
VALIDATION_SEVEREDR_DIR = "/content/kaggle/working/Data/validation/4/"


In [ ]:
split_size = .85
split_data(HEALTHY_SOURCE_DIR, TRAINING_HEALTHY_DIR, VALIDATION_HEALTHY_DIR, split_size)
split_data(MILDDR_SOURCE_DIR, TRAINING_MILDDR_DIR, VALIDATION_MILDDR_DIR, split_size)
split_data(MODERATEDR_SOURCE_DIR, TRAINING_MODERATEDR_DIR, VALIDATION_MODERATEDR_DIR, split_size)
split_data(PROLIFERATEDR_SOURCE_DIR, TRAINING_PROLIFERATEDR_DIR, VALIDATION_PROLIFERATEDR_DIR, split_size)
split_data(SEVEREDR_SOURCE_DIR, TRAINING_SEVEREDR_DIR, VALIDATION_SEVEREDR_DIR, split_size)

In [ ]:
TRAINING_DIR = '/content/kaggle/working/Data/training'
VALIDATION_DIR = '/content/kaggle/working/Data/validation'
healthy_imgs = len(os.listdir('/content/kaggle/working/Data/training/0')) + len(os.listdir('/content/kaggle/working/Data/validation/0'))
dr_imgs = len(os.listdir('/content/kaggle/working/Data/training/3')) + len(os.listdir('/content/kaggle/working/Data/validation/3'))
healthy_imgs, dr_imgs

(25810, 25810)

In [ ]:
def preprocessing_fn(img):
    image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    image = cv2.addWeighted(image,3.1, cv2.GaussianBlur(image, (15,15) ,IMG_SIZE/10),-3.5 ,128)
    adjusted = cv2.convertScaleAbs(image, alpha=1.8, beta=1.5)
    adjusted = cv2.resize(adjusted, (IMG_SIZE, IMG_SIZE))
    return adjusted.astype('float64')

In [ ]:
img_list_1 = [name for name in os.listdir(TRAINING_MILDDR_DIR)]
lim = len(img_list_1)
left_img_list_1 = [name for name in img_list_1 if 'left' in name]
right_img_list_1 = [name for name in img_list_1 if 'right' in name]

In [ ]:
def merge_fn(DIR, img_list2, img):
    l = len(img_list2)
    index = random.randint(0,l-1)
    img2 = cv2.imread(DIR + img_list2[index])
    img2 = cv2.addWeighted(img, 0.5, img2, 0.5, 1)
    return img2


In [ ]:
img = cv2.imread(TRAINING_MILDDR_DIR + left_img_list_1[5])
test = merge_fn(TRAINING_MILDDR_DIR, left_img_list_1, img)
plt.imshow(test)

In [ ]:
def oversampling(DIR,n):
    img_list = [name for name in os.listdir(DIR)]
    lim = len(img_list)
    left_img_list = [name for name in img_list if 'left' in name]
    right_img_list = [name for name in img_list if 'right' in name]

    for i in range(n):
        index = random.randint(0,lim-1)
        fn = img_list[index]
        file_path = DIR + fn

        if os.path.getsize(file_path)>0:
            img = cv2.imread(file_path)
            r = random.randint(0,2)

            if r==0:
                flip = cv2.flip(img,0)
                cv2.imwrite(file_path+str(i)+'.jpeg',flip)
            elif r==1:
                if 'left' in fn:
                    merge = merge_fn(DIR, left_img_list, img)
                else:
                    merge = merge_fn(DIR, right_img_list, img)
                cv2.imwrite(file_path+str(i)+'.jpeg',merge)
            else:
                cv2.imwrite(file_path+str(i)+'.jpeg',img)

len_healthy = len(os.listdir(TRAINING_HEALTHY_DIR))
len_dr = len(os.listdir(TRAINING_MILDDR_DIR))
oversampling(TRAINING_MILDDR_DIR, len_healthy - len_dr)

len_dr = len(os.listdir(TRAINING_MODERATEDR_DIR))
oversampling(TRAINING_MODERATEDR_DIR, len_healthy - len_dr)

len_dr = len(os.listdir(TRAINING_PROLIFERATEDR_DIR))
oversampling(TRAINING_PROLIFERATEDR_DIR, len_healthy - len_dr)

len_dr = len(os.listdir(TRAINING_SEVEREDR_DIR))
oversampling(TRAINING_SEVEREDR_DIR, len_healthy - len_dr)

In [ ]:
def validation_oversampler(DIR, n):
    img_list = [name for name in os.listdir(DIR)]
    lim = len(img_list)
    left_img_list = [name for name in img_list if 'left' in name]
    right_img_list = [name for name in img_list if 'right' in name]
    for i in range(n):
        index = random.randint(0,lim-1)
        fn = img_list[index]
        file_path = DIR + fn

        if os.path.getsize(file_path)>0:
            img = cv2.imread(file_path)
            r = random.randint(0,2)

            if r==0:
                flip = cv2.flip(img,0)
                cv2.imwrite(file_path+str(i)+'.jpeg',flip)
            else:
                cv2.imwrite(file_path+str(i)+'.jpeg',img)

len_healthy = len(os.listdir(VALIDATION_HEALTHY_DIR))

len_dr = len(os.listdir(VALIDATION_MILDDR_DIR))
validation_oversampler(VALIDATION_MILDDR_DIR, len_healthy - len_dr)

len_dr = len(os.listdir(VALIDATION_MODERATEDR_DIR))
oversampling(VALIDATION_MODERATEDR_DIR, len_healthy - len_dr)

len_dr = len(os.listdir(VALIDATION_PROLIFERATEDR_DIR))
oversampling(VALIDATION_PROLIFERATEDR_DIR, len_healthy - len_dr)

len_dr = len(os.listdir(VALIDATION_SEVEREDR_DIR))
oversampling(VALIDATION_SEVEREDR_DIR, len_healthy - len_dr)

In [ ]:
len(os.listdir(TRAINING_MODERATEDR_DIR)), len(os.listdir(TRAINING_SEVEREDR_DIR))

(21938, 21938)

In [ ]:
def train_val_generators(TRAINING_DIR, VALIDATION_DIR):
    train_datagen = ImageDataGenerator(rescale=1./255,
                                       horizontal_flip=True,
                                      preprocessing_function=preprocessing_fn)

    train_generator = train_datagen.flow_from_directory(directory=TRAINING_DIR,
                                                        batch_size=64,
                                                        class_mode='categorical',
                                                        target_size=(IMG_SIZE, IMG_SIZE))

    validation_datagen = ImageDataGenerator(rescale=1./255,
                                            horizontal_flip=True,
                                            preprocessing_function=preprocessing_fn)
    validation_generator = validation_datagen.flow_from_directory(directory=VALIDATION_DIR,
                                                                batch_size=32,
                                                                class_mode='categorical',
                                                                target_size=(IMG_SIZE, IMG_SIZE))

    return train_generator, validation_generator

train_generator, validation_generator = train_val_generators(TRAINING_DIR, VALIDATION_DIR)

Found 109690 images belonging to 5 classes.
Found 19360 images belonging to 5 classes.


## **Ensembling the models by taking the weighted sum**

In [ ]:
model1 = keras.models.load_model('/content/DenseNet')
model2 = keras.models.load_model('/content/MobileNet')
model3 = keras.models.load_model('/content/VGG')

In [ ]:
#MobileNet val - 50.24
#DenseNet - 53.88
#VGG - 47.70
s = 50.24 + 53.88 + 47.70
n_batches = 19360/32
n = 19360/32
tot_error = 0
m1_weight = 50.24/s
m2_weight = 53.88/s
m3_weight = 47.70/s
for imgs,labels in validation_generator:
  n_batches-=1
  if n_batches>0:
    ypred1 = model1.predict(imgs)*m1_weight
    ypred2 = model2.predict(imgs)*m2_weight
    ypred3 = model3.predict(imgs)*m3_weight
    gc.collect()
    final_preds = ypred1 + ypred2 + ypred3
    pred_class = keras.backend.argmax(final_preds, axis=1)
    label2 = keras.backend.argmax(labels, axis=1)
    acc = tf.reduce_mean(tf.cast(tf.math.equal(label2,pred_class),'float32'))
    print(acc, n_batches)
    tot_acc+=acc
  else:
    break
tot_acc/=n

1/1 [==============================] - 5s 5s/step
tf.Tensor(0.75, shape=(), dtype=float32)
1/1 [==============================] - 0s 48ms/step
tf.Tensor(0.8125, shape=(), dtype=float32)
1/1 [==============================] - 0s 36ms/step
tf.Tensor(0.8125, shape=(), dtype=float32)
1/1 [==============================] - 0s 26ms/step
tf.Tensor(0.78125, shape=(), dtype=float32)
1/1 [==============================] - 0s 35ms/step
tf.Tensor(0.71875, shape=(), dtype=float32)
1/1 [==============================] - 0s 29ms/step
tf.Tensor(0.65625, shape=(), dtype=float32)
1/1 [==============================] - 0s 33ms/step
tf.Tensor(0.625, shape=(), dtype=float32)
1/1 [==============================] - 0s 27ms/step
tf.Tensor(0.59375, shape=(), dtype=float32)
1/1 [==============================] - 0s 35ms/step
tf.Tensor(0.75, shape=(), dtype=float32)
1/1 [==============================] - 0s 28ms/step
tf.Tensor(0.8125, shape=(), dtype=float32)
1/1 [==============================] - 0s 33ms/step
t

In [ ]:
tot_acc

<tf.Tensor: shape=(), dtype=float32, numpy=0.7148244>

By ensembling three models, we have achieved a validation accuracy of **0.7148244**